In [1]:
import numpy as np
import sys
sys.path.append('.')
from step12_main import *

In [2]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1) # ys tuple
print(y.data)

In [3]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1) # ys tuple
print(y.data)

5
